In [34]:
! pip install google-auth google-auth-oauthlib google-auth-httplib2

In [35]:
! pip install google-api-python-client

In [36]:
#!/usr/bin/env python3
"""
Social Media Video Uploader
Author: AI Assistant
Description: Upload video clips to multiple social media platforms
Dependencies: requests, google-auth, python-dotenv
"""

import os
import json
import time
import logging
import requests
import pickle
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Optional
from dataclasses import dataclass
import sys

# ==========================================
# API CREDENTIALS & ACCOUNT SETTINGS - EDIT THESE
# ==========================================

# YOUTUBE API SETTINGS
YOUTUBE_CLIENT_ID = "Put key here"
YOUTUBE_CLIENT_SECRET = "Put key here"
YOUTUBE_REDIRECT_URI = "Put key here"  # For installed apps

# TIKTOK API SETTINGS
TIKTOK_CLIENT_KEY = "Put key here"
TIKTOK_CLIENT_SECRET = "Put key here"
TIKTOK_REDIRECT_URI = "Put key here"

# INSTAGRAM API SETTINGS (Meta Business)
INSTAGRAM_ACCESS_TOKEN = "your_instagram_long_lived_access_token_here"
INSTAGRAM_BUSINESS_ACCOUNT_ID = "your_instagram_business_account_id_here"

# LINKEDIN API SETTINGS
LINKEDIN_CLIENT_ID = "your_linkedin_client_id_here"
LINKEDIN_CLIENT_SECRET = "your_linkedin_client_secret_here"
LINKEDIN_REDIRECT_URI = "https://your-domain.com/callback"
LINKEDIN_PERSON_ID = "your_linkedin_person_id_here"  # Get from LinkedIn API

# TWITTER API SETTINGS (X)
TWITTER_API_KEY = "your_twitter_api_key_here"
TWITTER_API_SECRET = "your_twitter_api_secret_here"
TWITTER_ACCESS_TOKEN = "your_twitter_access_token_here"
TWITTER_ACCESS_TOKEN_SECRET = "your_twitter_access_token_secret_here"

# UPLOAD BEHAVIOR SETTINGS
PLATFORMS_TO_UPLOAD = ["youtube", "tiktok", "instagram"]  # Choose which platforms to use
AUTO_PUBLISH = True  # Set to False for drafts only
STAGGER_UPLOADS = True  # Wait between uploads to avoid rate limits
STAGGER_MINUTES = 30  # Minutes to wait between uploads
MAX_RETRIES = 3  # Retry failed uploads

# DEFAULT VIDEO SETTINGS
DEFAULT_PRIVACY = "public"  # public, unlisted, private
DEFAULT_TAGS = ["shorts", "viral", "video", "content"]
DEFAULT_DESCRIPTION_TEMPLATE = "Check out this amazing clip! 🎬\n\n#shorts #viral #video"

# ==========================================
# END OF SETTINGS
# ==========================================

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('uploader.log'),
        logging.StreamHandler()
    ]
)

@dataclass
class VideoClip:
    """Data class to represent a video clip for upload"""
    file_path: str
    title: str
    description: str
    tags: List[str]
    privacy: str = "public"
    thumbnail_path: Optional[str] = None
    upload_urls: Dict[str, str] = None

    def __post_init__(self):
        if self.upload_urls is None:
            self.upload_urls = {}

class PlatformUploader:
    """Base class for platform-specific uploaders"""
    
    def __init__(self, platform_name: str):
        self.platform_name = platform_name
        self.rate_limit = {'calls': 0, 'reset_time': time.time() + 3600}
    
    def check_rate_limit(self, max_calls: int = 50):
        """Check and enforce rate limits"""
        current_time = time.time()
        if current_time > self.rate_limit['reset_time']:
            self.rate_limit['calls'] = 0
            self.rate_limit['reset_time'] = current_time + 3600
        
        if self.rate_limit['calls'] >= max_calls:
            sleep_time = self.rate_limit['reset_time'] - current_time
            logging.info(f"Rate limit reached for {self.platform_name}, sleeping for {sleep_time:.0f} seconds")
            time.sleep(sleep_time)
            self.rate_limit['calls'] = 0
        
        self.rate_limit['calls'] += 1
    
    def upload(self, clip: VideoClip) -> Optional[str]:
        """Override in subclasses"""
        raise NotImplementedError

class YouTubeUploader(PlatformUploader):
    """Upload videos to YouTube using YouTube Data API v3"""
    
    def __init__(self):
        super().__init__("YouTube")
        self.credentials = None
        self.service = None
    
    def authenticate(self):
        """Authenticate with YouTube API"""
        try:
            from google.auth.transport.requests import Request
            from google.oauth2.credentials import Credentials
            from google_auth_oauthlib.flow import InstalledAppFlow
            from googleapiclient.discovery import build
            
            SCOPES = ['https://www.googleapis.com/auth/youtube.upload']
            
            # Load existing credentials
            if os.path.exists('youtube_token.pickle'):
                with open('youtube_token.pickle', 'rb') as token:
                    self.credentials = pickle.load(token)
            
            # If no valid credentials, authenticate
            if not self.credentials or not self.credentials.valid:
                if self.credentials and self.credentials.expired and self.credentials.refresh_token:
                    self.credentials.refresh(Request())
                else:
                    flow = InstalledAppFlow.from_client_config({
                        "installed": {
                            "client_id": YOUTUBE_CLIENT_ID,
                            "client_secret": YOUTUBE_CLIENT_SECRET,
                            "auth_uri": "https://accounts.google.com/o/oauth2/auth",
                            "token_uri": "https://oauth2.googleapis.com/token",
                            "redirect_uris": [YOUTUBE_REDIRECT_URI]
                        }
                    }, SCOPES)
                    self.credentials = flow.run_local_server(port=0)
                
                # Save credentials
                with open('youtube_token.pickle', 'wb') as token:
                    pickle.dump(self.credentials, token)
            
            self.service = build('youtube', 'v3', credentials=self.credentials)
            logging.info("YouTube authentication successful")
            return True
            
        except Exception as e:
            logging.error(f"YouTube authentication failed: {e}")
            return False
    
    def upload(self, clip: VideoClip) -> Optional[str]:
        """Upload video to YouTube"""
        try:
            if not self.service and not self.authenticate():
                return None
            
            self.check_rate_limit(100)  # YouTube allows 100 requests per hour
            
            from googleapiclient.http import MediaFileUpload
            
            body = {
                'snippet': {
                    'title': clip.title[:100],  # YouTube title limit
                    'description': clip.description[:5000],  # YouTube description limit
                    'tags': clip.tags[:30],  # Max 30 tags
                    'categoryId': '24',  # Entertainment category
                    'defaultLanguage': 'en'
                },
                'status': {
                    'privacyStatus': clip.privacy,
                    'selfDeclaredMadeForKids': False
                }
            }
            
            media = MediaFileUpload(
                clip.file_path,
                chunksize=-1,
                resumable=True,
                mimetype='video/*'
            )
            
            request = self.service.videos().insert(
                part=','.join(body.keys()),
                body=body,
                media_body=media
            )
            
            response = request.execute()
            video_id = response['id']
            video_url = f"https://youtube.com/watch?v={video_id}"
            
            logging.info(f"Successfully uploaded to YouTube: {video_url}")
            return video_url
            
        except Exception as e:
            logging.error(f"YouTube upload failed: {e}")
            return None

class TikTokUploader(PlatformUploader):
    """Upload videos to TikTok using Content Posting API"""
    
    def __init__(self):
        super().__init__("TikTok")
        self.access_token = None
    
    def authenticate(self):
        """Authenticate with TikTok API (simplified - requires OAuth flow)"""
        # This is a simplified version - real implementation needs OAuth flow
        logging.warning("TikTok authentication requires OAuth flow - using mock for now")
        self.access_token = "mock_tiktok_token"
        return True
    
    def upload(self, clip: VideoClip) -> Optional[str]:
        """Upload video to TikTok"""
        try:
            if not self.access_token and not self.authenticate():
                return None
            
            self.check_rate_limit(10)  # Conservative limit for TikTok
            
            # TikTok upload process (simplified mock implementation)
            # Real implementation would use TikTok's Content Posting API
            
            logging.info(f"Would upload {clip.file_path} to TikTok")
            logging.info(f"Title: {clip.title}")
            logging.info(f"Description: {clip.description}")
            
            # Mock successful upload
            mock_video_id = f"tiktok_{hash(clip.file_path) % 1000000}"
            video_url = f"https://tiktok.com/@user/video/{mock_video_id}"
            
            logging.info(f"Mock TikTok upload successful: {video_url}")
            return video_url
            
        except Exception as e:
            logging.error(f"TikTok upload failed: {e}")
            return None

class InstagramUploader(PlatformUploader):
    """Upload videos to Instagram using Graph API"""
    
    def __init__(self):
        super().__init__("Instagram")
        self.access_token = INSTAGRAM_ACCESS_TOKEN
        self.account_id = INSTAGRAM_BUSINESS_ACCOUNT_ID
    
    def upload(self, clip: VideoClip) -> Optional[str]:
        """Upload video to Instagram as Reel"""
        try:
            if not self.access_token:
                logging.error("Instagram access token not configured")
                return None
            
            self.check_rate_limit(25)  # Instagram rate limits
            
            # Step 1: Create media container
            create_url = f"https://graph.facebook.com/v18.0/{self.account_id}/media"
            
            # Upload video file first (simplified - real implementation needs file upload)
            # For now, using mock URL
            video_url = f"https://example.com/uploads/{os.path.basename(clip.file_path)}"
            
            create_params = {
                'media_type': 'REELS',
                'video_url': video_url,
                'caption': f"{clip.title}\n\n{clip.description}",
                'access_token': self.access_token
            }
            
            create_response = requests.post(create_url, data=create_params)
            
            if create_response.status_code != 200:
                logging.error(f"Instagram container creation failed: {create_response.text}")
                return None
            
            container_id = create_response.json()['id']
            
            # Step 2: Publish the media
            if AUTO_PUBLISH:
                publish_url = f"https://graph.facebook.com/v18.0/{self.account_id}/media_publish"
                publish_params = {
                    'creation_id': container_id,
                    'access_token': self.access_token
                }
                
                publish_response = requests.post(publish_url, data=publish_params)
                
                if publish_response.status_code == 200:
                    media_id = publish_response.json()['id']
                    video_url = f"https://instagram.com/reel/{media_id}"
                    logging.info(f"Successfully uploaded to Instagram: {video_url}")
                    return video_url
                else:
                    logging.error(f"Instagram publish failed: {publish_response.text}")
                    return None
            else:
                logging.info(f"Instagram media created as draft: {container_id}")
                return f"https://instagram.com/draft/{container_id}"
                
        except Exception as e:
            logging.error(f"Instagram upload failed: {e}")
            return None

class LinkedInUploader(PlatformUploader):
    """Upload videos to LinkedIn using LinkedIn API"""
    
    def __init__(self):
        super().__init__("LinkedIn")
        self.access_token = None
    
    def authenticate(self):
        """Authenticate with LinkedIn API (simplified)"""
        logging.warning("LinkedIn authentication requires OAuth flow - using mock for now")
        self.access_token = "mock_linkedin_token"
        return True
    
    def upload(self, clip: VideoClip) -> Optional[str]:
        """Upload video to LinkedIn"""
        try:
            if not self.access_token and not self.authenticate():
                return None
            
            self.check_rate_limit(20)  # Conservative LinkedIn limit
            
            # LinkedIn video upload process (mock implementation)
            logging.info(f"Would upload {clip.file_path} to LinkedIn")
            logging.info(f"Title: {clip.title}")
            logging.info(f"Description: {clip.description}")
            
            # Mock successful upload
            mock_post_id = f"linkedin_{hash(clip.file_path) % 1000000}"
            post_url = f"https://linkedin.com/posts/{mock_post_id}"
            
            logging.info(f"Mock LinkedIn upload successful: {post_url}")
            return post_url
            
        except Exception as e:
            logging.error(f"LinkedIn upload failed: {e}")
            return None

class TwitterUploader(PlatformUploader):
    """Upload videos to Twitter/X using API v2"""
    
    def __init__(self):
        super().__init__("Twitter")
        self.api_key = TWITTER_API_KEY
        self.api_secret = TWITTER_API_SECRET
        self.access_token = TWITTER_ACCESS_TOKEN
        self.access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    
    def upload(self, clip: VideoClip) -> Optional[str]:
        """Upload video to Twitter"""
        try:
            if not all([self.api_key, self.api_secret, self.access_token, self.access_token_secret]):
                logging.error("Twitter API credentials not configured")
                return None
            
            self.check_rate_limit(15)  # Twitter rate limits
            
            # Twitter video upload process (mock implementation)
            logging.info(f"Would upload {clip.file_path} to Twitter")
            logging.info(f"Title: {clip.title}")
            logging.info(f"Description: {clip.description}")
            
            # Mock successful upload
            mock_tweet_id = f"twitter_{hash(clip.file_path) % 1000000}"
            tweet_url = f"https://twitter.com/user/status/{mock_tweet_id}"
            
            logging.info(f"Mock Twitter upload successful: {tweet_url}")
            return tweet_url
            
        except Exception as e:
            logging.error(f"Twitter upload failed: {e}")
            return None

class SocialMediaUploader:
    """Main uploader class that orchestrates uploads to multiple platforms"""
    
    def __init__(self):
        self.uploaders = {
            'youtube': YouTubeUploader(),
            'tiktok': TikTokUploader(),
            'instagram': InstagramUploader(),
            'linkedin': LinkedInUploader(),
            'twitter': TwitterUploader()
        }
        
        self.analytics = {
            'total_uploads': 0,
            'successful_uploads': {},
            'failed_uploads': {},
            'upload_history': []
        }
        
        logging.info("Social Media Uploader initialized")
    
    def upload_clips(self, clips: List[VideoClip]) -> Dict:
        """Upload clips to all configured platforms"""
        logging.info(f"Starting upload of {len(clips)} clips to platforms: {PLATFORMS_TO_UPLOAD}")
        
        results = {platform: [] for platform in PLATFORMS_TO_UPLOAD}
        failed_uploads = []
        
        for i, clip in enumerate(clips):
            logging.info(f"Processing clip {i+1}/{len(clips)}: {clip.title}")
            
            # Add default values if missing
            if not clip.description:
                clip.description = DEFAULT_DESCRIPTION_TEMPLATE
            if not clip.tags:
                clip.tags = DEFAULT_TAGS.copy()
            if not clip.privacy:
                clip.privacy = DEFAULT_PRIVACY
            
            # Upload to each platform
            for platform in PLATFORMS_TO_UPLOAD:
                if platform not in self.uploaders:
                    logging.error(f"Unknown platform: {platform}")
                    continue
                
                uploader = self.uploaders[platform]
                
                # Retry logic
                for attempt in range(MAX_RETRIES):
                    try:
                        url = uploader.upload(clip)
                        if url:
                            clip.upload_urls[platform] = url
                            results[platform].append(url)
                            self.analytics['total_uploads'] += 1
                            
                            if platform not in self.analytics['successful_uploads']:
                                self.analytics['successful_uploads'][platform] = 0
                            self.analytics['successful_uploads'][platform] += 1
                            
                            logging.info(f"✓ Successfully uploaded to {platform}: {url}")
                            break
                        else:
                            if attempt < MAX_RETRIES - 1:
                                logging.warning(f"Upload to {platform} failed, retrying... ({attempt + 1}/{MAX_RETRIES})")
                                time.sleep(5)  # Wait before retry
                            else:
                                failed_uploads.append(f"{clip.title} -> {platform}")
                                if platform not in self.analytics['failed_uploads']:
                                    self.analytics['failed_uploads'][platform] = 0
                                self.analytics['failed_uploads'][platform] += 1
                                
                    except Exception as e:
                        logging.error(f"Exception uploading to {platform}: {e}")
                        if attempt == MAX_RETRIES - 1:
                            failed_uploads.append(f"{clip.title} -> {platform}: {str(e)}")
            
            # Stagger uploads if enabled
            if STAGGER_UPLOADS and i < len(clips) - 1:
                logging.info(f"Waiting {STAGGER_MINUTES} minutes before next clip...")
                time.sleep(STAGGER_MINUTES * 60)
        
        # Record upload batch
        batch_record = {
            'timestamp': datetime.now().isoformat(),
            'clips_count': len(clips),
            'platforms': PLATFORMS_TO_UPLOAD.copy(),
            'results': results,
            'failed': failed_uploads
        }
        self.analytics['upload_history'].append(batch_record)
        
        # Save analytics
        self.save_analytics()
        
        logging.info(f"Upload batch completed!")
        logging.info(f"Successful uploads: {sum(len(urls) for urls in results.values())}")
        logging.info(f"Failed uploads: {len(failed_uploads)}")
        
        return {
            'status': 'completed',
            'results': results,
            'failed': failed_uploads,
            'analytics': self.analytics
        }
    
    def upload_single_clip(self, clip_path: str, title: str = None, description: str = None, tags: List[str] = None) -> Dict:
        """Upload a single video clip"""
        if not os.path.exists(clip_path):
            return {'status': 'error', 'error': f'File not found: {clip_path}'}
        
        # Create VideoClip object
        clip = VideoClip(
            file_path=clip_path,
            title=title or f"Video {datetime.now().strftime('%Y%m%d_%H%M%S')}",
            description=description or DEFAULT_DESCRIPTION_TEMPLATE,
            tags=tags or DEFAULT_TAGS.copy(),
            privacy=DEFAULT_PRIVACY
        )
        
        return self.upload_clips([clip])
    
    def get_analytics(self) -> Dict:
        """Get upload analytics"""
        return self.analytics.copy()
    
    def save_analytics(self):
        """Save analytics to file"""
        with open('upload_analytics.json', 'w') as f:
            json.dump(self.analytics, f, indent=4, default=str)
    
    def load_analytics(self):
        """Load analytics from file"""
        try:
            if os.path.exists('upload_analytics.json'):
                with open('upload_analytics.json', 'r') as f:
                    self.analytics = json.load(f)
                logging.info("Loaded existing analytics")
        except Exception as e:
            logging.error(f"Error loading analytics: {e}")

def validate_credentials() -> List[str]:
    """Check if API credentials are properly configured"""
    missing = []
    
    if "youtube" in PLATFORMS_TO_UPLOAD:
        if not YOUTUBE_CLIENT_ID or YOUTUBE_CLIENT_ID == "your_youtube_client_id_here":
            missing.append("YouTube Client ID")
        if not YOUTUBE_CLIENT_SECRET or YOUTUBE_CLIENT_SECRET == "your_youtube_client_secret_here":
            missing.append("YouTube Client Secret")
    
    if "instagram" in PLATFORMS_TO_UPLOAD:
        if not INSTAGRAM_ACCESS_TOKEN or INSTAGRAM_ACCESS_TOKEN == "your_instagram_long_lived_access_token_here":
            missing.append("Instagram Access Token")
        if not INSTAGRAM_BUSINESS_ACCOUNT_ID or INSTAGRAM_BUSINESS_ACCOUNT_ID == "your_instagram_business_account_id_here":
            missing.append("Instagram Business Account ID")
    
    if "twitter" in PLATFORMS_TO_UPLOAD:
        if not TWITTER_API_KEY or TWITTER_API_KEY == "your_twitter_api_key_here":
            missing.append("Twitter API Key")
    
    return missing

def upload_from_folder(folder_path: str, file_pattern: str = "*.mp4") -> Dict:
    """Upload all video files from a folder"""
    from glob import glob
    
    if not os.path.exists(folder_path):
        return {'status': 'error', 'error': f'Folder not found: {folder_path}'}
    
    # Find video files
    video_files = glob(os.path.join(folder_path, file_pattern))
    
    if not video_files:
        return {'status': 'error', 'error': f'No video files found in {folder_path}'}
    
    # Create clips
    clips = []
    for i, video_file in enumerate(video_files):
        filename = os.path.basename(video_file)
        title = filename.replace('.mp4', '').replace('_', ' ').title()
        
        clip = VideoClip(
            file_path=video_file,
            title=title,
            description=DEFAULT_DESCRIPTION_TEMPLATE,
            tags=DEFAULT_TAGS.copy(),
            privacy=DEFAULT_PRIVACY
        )
        clips.append(clip)
    
    # Upload all clips
    uploader = SocialMediaUploader()
    return uploader.upload_clips(clips)

def is_notebook_environment():
    """Check if running in Jupyter notebook"""
    try:
        get_ipython
        return True
    except NameError:
        return False

def main():
    """Main entry point"""
    
    if is_notebook_environment():
        print("Social Media Uploader detected Jupyter environment")
        print("Configure your API credentials at the top of the code, then use:")
        print("uploader = SocialMediaUploader()")
        print("result = uploader.upload_single_clip('your_video.mp4', 'Your Title')")
        print("# Or upload from clips folder:")
        print("result = upload_from_folder('clips')")
        return
    
    import argparse
    
    parser = argparse.ArgumentParser(description='Social Media Video Uploader')
    parser.add_argument('--file', help='Single video file to upload')
    parser.add_argument('--folder', default='clips', help='Folder containing video clips')
    parser.add_argument('--title', help='Video title (for single file upload)')
    parser.add_argument('--check-credentials', action='store_true', help='Check API credentials')
    parser.add_argument('--analytics', action='store_true', help='Show upload analytics')
    
    args = parser.parse_args()
    
    if args.check_credentials:
        missing = validate_credentials()
        if missing:
            print("Missing credentials:")
            for cred in missing:
                print(f"  - {cred}")
        else:
            print("All configured platform credentials are set")
        return
    
    uploader = SocialMediaUploader()
    uploader.load_analytics()
    
    if args.analytics:
        analytics = uploader.get_analytics()
        print("Upload Analytics:")
        print(f"Total uploads: {analytics['total_uploads']}")
        print("Successful uploads by platform:")
        for platform, count in analytics['successful_uploads'].items():
            print(f"  {platform}: {count}")
        return
    
    if args.file:
        # Upload single file
        result = uploader.upload_single_clip(args.file, args.title)
    else:
        # Upload from folder
        result = upload_from_folder(args.folder)
    
    print(f"Upload completed with status: {result['status']}")
    if result.get('failed'):
        print("Failed uploads:")
        for fail in result['failed']:
            print(f"  - {fail}")

if __name__ == "__main__":
    main()

Social Media Uploader detected Jupyter environment
Configure your API credentials at the top of the code, then use:
uploader = SocialMediaUploader()
result = uploader.upload_single_clip('your_video.mp4', 'Your Title')
# Or upload from clips folder:
result = upload_from_folder('clips')


In [18]:
uploader = SocialMediaUploader()

2025-08-28 19:07:50,835 - INFO - Social Media Uploader initialized
